In [1]:
import pandas as pd
import numpy as np
import itertools

from xgboost import XGBClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report,\
precision_recall_curve, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, train_test_split

**Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)**

In [2]:
df = pd.read_csv("Churn_Modelling.csv")
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [3]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [5]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [6]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [7]:


gender = Pipeline([
                ('selector', FeatureSelector(column='Gender')),
                ('ohe', OHEEncoder(key='Gender'))
            ])
gender.fit(X_train)
gender.transform(X_test).head(3)

,Gender_Female,Gender_Male
9394,1,0
898,1,0
2398,1,0


In [8]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
        ('selector', NumberSelector(key=cont_col)),
        ('scaler', MinMaxScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [9]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [10]:
rf_pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

In [11]:
xgb_pipeline = Pipeline([
    ('features',feats),
    ('classifier', XGBClassifier(random_state = 42)),
])

In [12]:
lr_pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

In [13]:
%%time
rf_pipeline.fit(X_train, y_train)

CPU times: user 597 ms, sys: 11.6 ms, total: 608 ms
Wall time: 609 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [14]:
%%time
xgb_pipeline.fit(X_train, y_train)

CPU times: user 1.05 s, sys: 3.96 ms, total: 1.05 s
Wall time: 307 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [15]:
%%time
lr_pipeline.fit(X_train, y_train)

CPU times: user 96.2 ms, sys: 24.2 ms, total: 120 ms
Wall time: 53.2 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [16]:
rf_preds = rf_pipeline.predict_proba(X_test)[:, 1]
xgb_preds = xgb_pipeline.predict_proba(X_test)[:, 1]
lr_preds = lr_pipeline.predict_proba(X_test)[:, 1]

In [17]:
rf_precision, rf_recall, rf_thresholds = precision_recall_curve(y_test, rf_preds)

rf_fscore = (2 * rf_precision * rf_recall) / (rf_precision + rf_recall)
# locate the index of the largest f score
ix_rf = np.argmax(rf_fscore)
print('RF best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (rf_thresholds[ix_rf], 
                                                                        rf_fscore[ix_rf],
                                                                        rf_precision[ix_rf],
                                                                        rf_recall[ix_rf]))

RF best Threshold=0.360000, F-Score=0.640, Precision=0.627, Recall=0.654


In [18]:
xgb_precision, xgb_recall, xgb_thresholds = precision_recall_curve(y_test, xgb_preds)

xgb_fscore = (2 * xgb_precision * xgb_recall) / (xgb_precision + xgb_recall)
# locate the index of the largest f score
ix_xgb = np.argmax(xgb_fscore)
print('XGB best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (xgb_thresholds[ix_xgb], 
                                                                        xgb_fscore[ix_xgb],
                                                                        xgb_precision[ix_xgb],
                                                                        xgb_recall[ix_xgb]))

XGB best Threshold=0.373318, F-Score=0.626, Precision=0.619, Recall=0.633


In [19]:
lr_precision, lr_recall, lr_thresholds = precision_recall_curve(y_test, lr_preds)

lr_fscore = (2 * lr_precision * lr_recall) / (lr_precision + lr_recall)
# locate the index of the largest f score
ix_lr = np.argmax(lr_fscore)
print('LR best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (lr_thresholds[ix_lr], 
                                                                        lr_fscore[ix_lr],
                                                                        lr_precision[ix_lr],
                                                                        lr_recall[ix_lr]))

LR best Threshold=0.286079, F-Score=0.509, Precision=0.460, Recall=0.570


**Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)**

**Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.**

Думаю что можно ориентироваться на recall чтобы максимально эффективно потратить деньги на привлечение

In [20]:
tn, fp, fn, tp = confusion_matrix(y_test, np.where(xgb_preds > xgb_thresholds[ix_xgb], 1, 0)).ravel()

In [21]:
print(f'На отложенной выборке из {y_test.shape[0]} людей мы заработаем {2 * tp - (fp + tp)}$\n\
Правильно классифицировали {xgb_recall[ix_xgb] * 100:.2f}% людей')

На отложенной выборке из 2500 людей мы заработаем 123$
Правильно классифицировали 63.26% людей


**Провести подбор гиперпараметров лучшей модели по итогам 2-3**

In [22]:
params = {'classifier__max_depth': np.arange(1, 25)}

In [23]:
%%time
rs = RandomizedSearchCV(estimator=xgb_pipeline,
                        param_distributions=params,
                        scoring='recall',
                        cv=5,
                        n_jobs=-1)

rs.fit(X_train, y_train)

CPU times: user 3.27 s, sys: 112 ms, total: 3.38 s
Wall time: 17.1 s


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('features',
                                              FeatureUnion(transformer_list=[('Geography',
                                                                              Pipeline(steps=[('selector',
                                                                                               FeatureSelector(column='Geography')),
                                                                                              ('ohe',
                                                                                               OHEEncoder(key='Geography'))])),
                                                                             ('Gender',
                                                                              Pipeline(steps=[('selector',
                                                                                               FeatureSelector(column='Gender')),
                               

In [24]:
rs.best_params_

{'classifier__max_depth': 20}

In [25]:
xgb_pipeline.set_params(**rs.best_params_)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [26]:
%%time
xgb_pipeline.fit(X_train, y_train)

CPU times: user 2.83 s, sys: 16.3 ms, total: 2.84 s
Wall time: 753 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [27]:
xgb_fin_preds = rf_pipeline.predict_proba(X_test)[:, 1]

In [28]:
xgb_precision, xgb_recall, xgb_thresholds = precision_recall_curve(y_test, xgb_fin_preds)

xgb_fscore = (2 * xgb_precision * xgb_recall) / (xgb_precision + xgb_recall)
# locate the index of the largest f score
ix_xgb = np.argmax(xgb_fscore)
print('XGB best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (xgb_thresholds[ix_xgb], 
                                                                        xgb_fscore[ix_xgb],
                                                                        xgb_precision[ix_xgb],
                                                                        xgb_recall[ix_xgb]))

XGB best Threshold=0.360000, F-Score=0.640, Precision=0.627, Recall=0.654


**Еще раз провести оценку экономической эффективности**

In [29]:
tn, fp, fn, tp = confusion_matrix(y_test, np.where(xgb_fin_preds > xgb_thresholds[ix_xgb], 1, 0)).ravel()

In [30]:
print(f'На отложенной выборке из {y_test.shape[0]} людей мы заработаем {2 * tp - (fp + tp)}$\n\
Правильно классифицировали {xgb_recall[ix_xgb] * 100:.2f}% людей')

На отложенной выборке из 2500 людей мы заработаем 142$
Правильно классифицировали 65.42% людей
